In [10]:
from tabula import read_pdf # requiere instalar java: https://www.java.com/es/download/
import pandas as pd
from unidecode import unidecode
import re

inpath = "E:/OneDrive/Datasets/Tuboleta/"
pdf = read_pdf(inpath + "CodigosPostales.pdf", pages='all')
len(pdf)

82

In [11]:
def limpiar_texto(texto):
    # Eliminar valores numéricos utilizando expresiones regulares
    texto_mayuscula = str.upper(texto)   # str. se utiliza en para tratar pd.Series

    # Remover tildes y caracteres especiales de entonacion
    texto_sin_tildes = unidecode(texto_mayuscula)

    # Quitar caracteres no alfabéticos y no espacios en blanco
    texto_limpio = re.sub(r'[^a-zA-Z\s]', '', texto_sin_tildes)

    # Aplicar trim para eliminar espacios en blanco al principio y al final
    texto_limpio = texto_limpio.strip()
    
    return texto_limpio

In [13]:
# Suponiendo que lista_dataframes es tu lista con los 82 DataFrames

# Concatenar los DataFrames de la lista
pdf_df = pd.concat(pdf, ignore_index=True)# 'ignore_index=True' reinicia los índices del DataFrame resultante
    # para que sean secuenciales, en lugar de conservar los índices originales de cada DataFrame

#################
# Descriptivos
print(pdf_df.head(5))
print(' ')
print('Dimensiones:')
print(pdf_df.shape)
print(' ')
print(pdf_df.columns)

  State / Departamento  Postal Code /\rCódigo Postal* City Name / Ciudad  \
0            Antioquia                          50001           Medellin   
1            Antioquia                          50002           Medellin   
2            Antioquia                          50003           Medellin   
3            Antioquia                          50004           Medellin   
4            Antioquia                          50005           Medellin   

  State\rCode  
0          AN  
1          AN  
2          AN  
3          AN  
4          AN  
 
Dimensiones:
(3681, 4)
 
Index(['State / Departamento', 'Postal Code /\rCódigo Postal*',
       'City Name / Ciudad', 'State\rCode'],
      dtype='object')


In [18]:
codpost = pdf_df.sort_values(by='State / Departamento', ascending=True).\
       rename(columns = {'State / Departamento': 'Departamento','Postal Code /\rCódigo Postal*': 'Codigo Postal','City Name / Ciudad': 'Ciudad','State\rCode': 'CodDep'}).\
       assign(Departamento = lambda x: x['Departamento'].apply(limpiar_texto),
              Ciudad = lambda x: x['Ciudad'].apply(limpiar_texto))
       
       #groupby('DEPARTAMENTO').agg({'COUNT': 'sum','PORCENTAJE': 'sum'}).reset_index().\


#################
# Descriptivos
print(codpost.head(5))
print(' ')
print('Dimensiones:')
print(codpost.shape)

     Departamento  Codigo Postal            Ciudad CodDep
3569     AMAZONAS         915019  PUERTO SANTANDER     AM
3575     AMAZONAS         917018        LA PEDRERA     AM
3574     AMAZONAS         917017        LA PEDRERA     AM
3573     AMAZONAS         917010        LA PEDRERA     AM
3572     AMAZONAS         916058    MIRITI  PARANA     AM
 
Dimensiones:
(3681, 4)


In [19]:
# Escritura
codpost.to_csv(inpath +'codpost.csv', 
               index=False, # El argumento 'index=False' evita que se incluyan los índices en el archivo CSV
               sep = '|',
               decimal = '.')  